In [1]:
!pip install git+https://github.com/openai/CLIP.git --quiet
!pip install timm ftfy regex tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.3 MB/s eta 0:00:

# Download task vectors

In [2]:
import subprocess
import sys

print("Downloading task_vectors source code...")

# Download just the source file we need
!wget -q https://raw.githubusercontent.com/mlfoundations/task_vectors/main/src/task_vectors.py -O task_vectors.py

print("✓ Downloaded task_vectors.py")

# Verify the file exists
import os
if os.path.exists('task_vectors.py'):
    print("✓ task_vectors.py is ready to use")
else:
    print("✗ Download failed")

✓ Downloaded task_vectors.py
✓ task_vectors.py is ready to use


In [3]:
# ============================================================================
# CELL 2: Import and test
# ============================================================================

# Now you can import directly
from task_vectors import TaskVector

print("✓ TaskVector imported successfully!")

# Test it
print("\nUsage:")
print("  task_vector = TaskVector('pretrained.pt', 'finetuned.pt')")
print("  enhancement = task_vector_balanced - task_vector_imbalanced")


✓ TaskVector imported successfully!

Usage:
  task_vector = TaskVector('pretrained.pt', 'finetuned.pt')
  enhancement = task_vector_balanced - task_vector_imbalanced


# imports

In [4]:
import gc

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
import clip 
import pandas as pd
import numpy as np
from PIL import Image
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Import TaskVector class
from task_vectors import TaskVector

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

PyTorch version: 2.6.0+cu124
CUDA available: True
Using device: cuda


# DMID dataset

In [6]:
CONFIG = {
    'clip_model': 'ViT-B/32',  # ← CHANGED: CLIP model instead of timm model
    'img_size': 224,
    'batch_size': 4,
    'epochs': 3,
    'lr': 1e-6,  # ← CHANGED: Lower LR for CLIP fine-tuning
    'alpha': 0.3,
    'seed': 42,
    'max_train_samples': 120
}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(CONFIG['seed'])
np.random.seed(CONFIG['seed'])
def empty_cache():
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    gc.collect()


In [7]:
base_path = '/kaggle/input/miniddsm2'

print("="*70)
print("LOADING MINI-DDSM DATASET")
print("="*70)

# USE JPEG-8 version (has Normal class)
dataset_version = 'MINI-DDSM-Complete-JPEG-8'
data_dir = Path(base_path) / dataset_version

print(f"✓ Using: {dataset_version}")
print(f"  Path: {data_dir}")

# Load metadata
metadata_file = data_dir / 'DataWMask.xlsx'

if metadata_file.exists():
    print(f"✓ Found metadata: {metadata_file.name}")
    df_meta = pd.read_excel(metadata_file)
    print(f"✓ Loaded {len(df_meta)} records")
    print(f"\nColumns: {list(df_meta.columns)}")
else:
    print("⚠ No metadata file found, will use folder structure only")
    df_meta = None


LOADING MINI-DDSM DATASET
✓ Using: MINI-DDSM-Complete-JPEG-8
  Path: /kaggle/input/miniddsm2/MINI-DDSM-Complete-JPEG-8
✓ Found metadata: DataWMask.xlsx
✓ Loaded 7808 records

Columns: ['fullPath', 'fileName', 'View', 'Side', 'Status', 'Tumour_Contour', 'Tumour_Contour2', 'Age', 'Density']


In [8]:
print(df_meta.head(3))

# Check Density (BI-RADS) distribution
print(f"\nBI-RADS (Density) Distribution:")
print(df_meta['Density'].value_counts().sort_index())

                            fullPath               fileName View   Side  \
0   Benign\0029\C_0029_1.LEFT_CC.jpg   C_0029_1.LEFT_CC.jpg   CC   LEFT   
1  Benign\0029\C_0029_1.LEFT_MLO.jpg  C_0029_1.LEFT_MLO.jpg  MLO   LEFT   
2  Benign\0029\C_0029_1.RIGHT_CC.jpg  C_0029_1.RIGHT_CC.jpg   CC  RIGHT   

   Status                          Tumour_Contour Tumour_Contour2   Age  \
0  Benign   Benign\0029\C_0029_1.LEFT_CC_Mask.jpg               -  66.0   
1  Benign  Benign\0029\C_0029_1.LEFT_MLO_Mask.jpg               -  66.0   
2  Benign                                       -               -  66.0   

   Density  
0        3  
1        3  
2        3  

BI-RADS (Density) Distribution:
Density
0       4
1    1076
2    3020
3    2320
4    1388
Name: count, dtype: int64


In [9]:
print("\n" + "="*70)
print("MAPPING IMAGES TO BI-RADS CATEGORIES")
print("="*70)

data = []

# Iterate through class folders
class_folders = {
    'Normal': data_dir / 'Normal',
    'Benign': data_dir / 'Benign',
    'Cancer': data_dir / 'Cancer'
}

for class_name, class_path in class_folders.items():
    if not class_path.exists():
        continue
    
    print(f"\nProcessing {class_name}...")
    
    # Find all case folders
    case_folders = [f for f in class_path.iterdir() if f.is_dir()]
    
    for case_folder in case_folders:
        case_id = case_folder.name
        
        # Find images (skip masks)
        images = [img for img in case_folder.glob('*.jpg') 
                  if 'mask' not in img.name.lower() or 'boundary' in img.name.lower()]
        
        for img_path in images:
            # Match with metadata using fileName or fullPath
            matching_rows = df_meta[df_meta['fileName'].str.contains(case_id, na=False)]
            
            if len(matching_rows) == 0:
                # Try fullPath match
                matching_rows = df_meta[df_meta['fullPath'].str.contains(case_id, na=False)]
            
            if len(matching_rows) > 0:
                row = matching_rows.iloc[0]
                birads = row['Density']
                age = row['Age']
                status = row['Status']
            else:
                # Fallback: infer from class
                birads = {'Normal': 1, 'Benign': 2, 'Cancer': 4}.get(class_name, 0)
                age = 50
                status = class_name
            
            data.append({
                'image_id': case_id,
                'image_path': str(img_path),
                'classification': class_name,
                'birads': birads,
                'age': age,
                'status': status
            })

df = pd.DataFrame(data)
print(f"\n✓ Mapped {len(df)} images")
print(f"\nBI-RADS Distribution:")
print(df['birads'].value_counts().sort_index())
print(f"\nClassification Distribution:")
print(df['classification'].value_counts())



MAPPING IMAGES TO BI-RADS CATEGORIES

Processing Normal...

Processing Benign...

Processing Cancer...

✓ Mapped 7808 images

BI-RADS Distribution:
birads
0       4
1    1168
2    3184
3    2260
4    1192
Name: count, dtype: int64

Classification Distribution:
classification
Cancer    2716
Benign    2684
Normal    2408
Name: count, dtype: int64


# Create Artificial Imbalance

In [10]:
import pandas as pd
import numpy as np

imbalance_ratio = 10  # ratio of majority to minority
max_samples_per_class = 300  # cap for majority classes to quickly prototype
np.random.seed(CONFIG['seed'])

# Downsample all classes
df_birads1 = df[df['birads'] == 1].sample(n=min(len(df[df['birads'] == 1]), max_samples_per_class), random_state=CONFIG['seed'])
df_birads2 = df[df['birads'] == 2].sample(n=min(len(df[df['birads'] == 2]), max_samples_per_class), random_state=CONFIG['seed'])
df_birads3 = df[df['birads'] == 3].sample(n=min(len(df[df['birads'] == 3]), max_samples_per_class), random_state=CONFIG['seed'])

# Make BI-RADS 4 scarce
birads4_target = max_samples_per_class // imbalance_ratio  # ~30 samples
df_birads4 = df[df['birads'] == 4].sample(n=min(len(df[df['birads'] == 4]), birads4_target), random_state=CONFIG['seed'])

# Combine and shuffle
df_downsampled = pd.concat([df_birads1, df_birads2, df_birads3, df_birads4]).sample(frac=1, random_state=CONFIG['seed']).reset_index(drop=True)
df_downsampled['birads_label'] = df_downsampled['birads'].astype(int) - 1

print('Downsampled counts:', df_downsampled['birads'].value_counts().sort_index())
print('Total samples:', len(df_downsampled))
df_downsampled
df = df_downsampled.copy()
df

Downsampled counts: birads
1    300
2    300
3    300
4     30
Name: count, dtype: int64
Total samples: 930


,image_id,image_path,classification,birads,age,status,birads_label
0,4167,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Benign,3,44.0,Benign,2
1,0144,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Cancer,3,74.0,Cancer,2
2,1850,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Cancer,1,82.0,Cancer,0
3,0003,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Normal,2,46.0,Cancer,1
4,2010,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Normal,1,74.0,Normal,0
...,...,...,...,...,...,...,...
925,0103,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Cancer,1,50.0,Cancer,0
926,0031,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Normal,1,81.0,Cancer,0
927,0598,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Normal,3,52.0,Normal,2
928,0271,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Normal,2,53.0,Benign,1


# Load CLIP and define prompts

In [20]:
# Load CLIP model and define prompts
print("="*70)
print("LOADING CLIP MODEL")
print("="*70)

# Load CLIP
model_clip, clip_preprocess = clip.load(CONFIG['clip_model'], device=device)

# CRITICAL FIX: Force FP32 to prevent NaN
model_clip = model_clip.float()

print(f"✓ Loaded CLIP {CONFIG['clip_model']}")
print(f"✓ Model dtype: {next(model_clip.parameters()).dtype}")
print(f"✓ Visual encoder parameters: {sum(p.numel() for p in model_clip.visual.parameters()):,}")

# Verify with test forward pass
test_img = torch.randn(2, 3, 224, 224).to(device)
with torch.no_grad():
    test_output = model_clip.encode_image(test_img)
    print(f"✓ Test forward pass: {test_output.shape}")
    if torch.isnan(test_output).any():
        print("❌ NaN detected - restart kernel required")
    else:
        print("✓ No NaN - ready to train!")


LOADING CLIP MODEL
✓ Loaded CLIP ViT-B/32
✓ Model dtype: torch.float32
✓ Visual encoder parameters: 87,849,216
✓ Test forward pass: torch.Size([2, 512])
✓ No NaN - ready to train!


In [22]:
# Define BI-RADS text prompts
birads_prompts = [
    "a mammogram showing BI-RADS category 1, normal breast tissue",
    "a mammogram showing BI-RADS category 2, benign findings",
    "a mammogram showing BI-RADS category 3, probably benign findings",
    "a mammogram showing BI-RADS category 4, suspicious abnormality"
]

print(f"✓ Defined {len(birads_prompts)} text prompts for BI-RADS categories")


✓ Defined 4 text prompts for BI-RADS categories


# Dataset class

In [23]:
# Contrastive Dataset class for CLIP fine-tuning
class BIRADSDataset(Dataset):
    def __init__(self, dataframe, preprocess, text_prompts):
        self.df = dataframe.reset_index(drop=True)
        self.preprocess = preprocess
        # Tokenize all prompts once
        self.text_tokens = clip.tokenize(text_prompts)
        print(f"✓ Tokenized {len(text_prompts)} text prompts")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try:
            # Load and preprocess image
            img = Image.open(row['image_path']).convert('RGB')
            img = self.preprocess(img)
            
            # Get corresponding text prompt
            label = row['birads_label']  # 0, 1, 2, or 3
            text = self.text_tokens[label]  # Matching text prompt
            
            return img, text, label
        except Exception as e:
            print(f"Error loading {row['image_path']}: {e}")
            return None

# Filtered dataset to skip None values
class FilteredDataset(Dataset):
    def __init__(self, base_dataset):
        self.base_ds = base_dataset
        self.valid_idx = [i for i in range(len(base_dataset)) 
                         if base_dataset[i] is not None]
        print(f"✓ Valid samples: {len(self.valid_idx)}/{len(base_dataset)}")
    
    def __len__(self):
        return len(self.valid_idx)
    
    def __getitem__(self, idx):
        return self.base_ds[self.valid_idx[idx]]

print("✓ Contrastive dataset classes defined")


✓ Contrastive dataset classes defined


In [24]:
# Prepare data splits with contrastive learning
print("="*70)
print("PREPARING DATA SPLITS")
print("="*70)

# Split data
train_df_5class, val_df_5class = train_test_split(
    df, 
    test_size=0.2, 
    stratify=df['birads_label'], 
    random_state=CONFIG['seed']
)

print(f"Split: Train={len(train_df_5class)}, Val={len(val_df_5class)}")

# Create datasets with text prompts
train_ds_5class = FilteredDataset(
    BIRADSDataset(train_df_5class, preprocess=clip_preprocess, text_prompts=birads_prompts)
)
val_ds_5class = FilteredDataset(
    BIRADSDataset(val_df_5class, preprocess=clip_preprocess, text_prompts=birads_prompts)
)

# Create DataLoaders
train_loader_5class = DataLoader(
    train_ds_5class, 
    batch_size=CONFIG['batch_size'], 
    shuffle=True, 
    num_workers=2
)
val_loader_5class = DataLoader(
    val_ds_5class, 
    batch_size=CONFIG['batch_size'], 
    num_workers=2
)

print(f"✓ Dataloaders ready: Train={len(train_ds_5class)}, Val={len(val_ds_5class)}")


PREPARING DATA SPLITS
Split: Train=744, Val=186
✓ Tokenized 4 text prompts
✓ Valid samples: 744/744
✓ Tokenized 4 text prompts
✓ Valid samples: 186/186
✓ Dataloaders ready: Train=744, Val=186


# Save pre-trained baseline

In [15]:
# Save pretrained CLIP visual encoder
print("="*70)
print("SAVING PRETRAINED BASELINE (CLIP visual encoder)")
print("="*70)

torch.save(model_clip.visual.state_dict(), 'clip_visual_pretrained.pt')

print(f"✓ Saved pretrained CLIP {CONFIG['clip_model']} visual encoder")
print(f"  Parameters: {sum(p.numel() for p in model_clip.visual.parameters()):,}")

empty_cache()


SAVING PRETRAINED BASELINE (CLIP visual encoder)
✓ Saved pretrained CLIP ViT-B/32 visual encoder
  Parameters: 87,849,216


# Prepare data for 5-class training


In [16]:
df

,image_id,image_path,classification,birads,age,status,birads_label
0,4167,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Benign,3,44.0,Benign,2
1,0144,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Cancer,3,74.0,Cancer,2
2,1850,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Cancer,1,82.0,Cancer,0
3,0003,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Normal,2,46.0,Cancer,1
4,2010,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Normal,1,74.0,Normal,0
...,...,...,...,...,...,...,...
925,0103,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Cancer,1,50.0,Cancer,0
926,0031,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Normal,1,81.0,Cancer,0
927,0598,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Normal,3,52.0,Normal,2
928,0271,/kaggle/input/miniddsm2/MINI-DDSM-Complete-JPE...,Normal,2,53.0,Benign,1


# Train Base Model (5-class on imbalanced data)

# Dry run: without freezing

In [25]:
# Train Base Model with Contrastive Loss - NO FREEZING
print("="*70)
print("TRAINING CLIP WITH CONTRASTIVE LOSS - ALL LAYERS TRAINABLE")
print("="*70)

# Make ALL parameters trainable (no freezing)
for param in model_clip.parameters():
    param.requires_grad = True

print("✓ ALL layers trainable (visual + text encoder)")
print(f"✓ Total trainable parameters: {sum(p.numel() for p in model_clip.parameters() if p.requires_grad):,}")

# Optimizer - training everything
optimizer_base = optim.AdamW(
    model_clip.parameters(),  # Train ALL parameters
    lr=1e-6,  # Conservative LR
    weight_decay=1e-4,
    eps=1e-8
)

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer_base, 
    T_max=CONFIG['epochs']
)

best_acc_base = 0

for epoch in range(CONFIG['epochs']):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1}/{CONFIG['epochs']}")
    print(f"{'='*70}")
    
    # ========== TRAINING ==========
    model_clip.train()
    train_loss = 0
    num_batches = 0
    nan_count = 0
    
    for batch_idx, (images, texts, labels) in enumerate(tqdm(train_loader_5class, desc="Training", leave=False)):
        images = images.to(device).float()  # Force FP32
        texts = texts.to(device)
        labels = labels.to(device)
        
        # Check inputs
        if torch.isnan(images).any() or torch.isinf(images).any():
            print(f"\n⚠ NaN/Inf in input images batch {batch_idx}")
            nan_count += 1
            continue
        
        optimizer_base.zero_grad()
        
        # Encode image and text
        image_features = model_clip.encode_image(images)
        text_features = model_clip.encode_text(texts)
        
        # Check for NaN after encoding
        if torch.isnan(image_features).any() or torch.isnan(text_features).any():
            print(f"\n⚠ NaN after encoding batch {batch_idx}")
            nan_count += 1
            if nan_count > 5:
                print("❌ Too many NaN batches, stopping epoch")
                break
            continue
        
        # Normalize features
        image_features = image_features / (image_features.norm(dim=-1, keepdim=True) + 1e-8)
        text_features = text_features / (text_features.norm(dim=-1, keepdim=True) + 1e-8)
        
        # Compute similarity (contrastive)
        logit_scale = model_clip.logit_scale.exp().clamp(max=100)
        logits_per_image = logit_scale * image_features @ text_features.T
        logits_per_text = logits_per_image.T
        
        # Contrastive loss: each image should match its corresponding text
        batch_size = images.shape[0]
        target = torch.arange(batch_size).to(device)
        
        loss_img = nn.CrossEntropyLoss()(logits_per_image, target)
        loss_txt = nn.CrossEntropyLoss()(logits_per_text, target)
        loss = (loss_img + loss_txt) / 2
        
        # Check loss
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"\n⚠ NaN/Inf loss batch {batch_idx}")
            nan_count += 1
            continue
        
        loss.backward()
        
        # Gradient clipping
        grad_norm = torch.nn.utils.clip_grad_norm_(
            model_clip.parameters(),  # Clip all gradients
            max_norm=1.0
        )
        
        optimizer_base.step()
        
        train_loss += loss.item()
        num_batches += 1
        
        # Progress check every 50 batches
        if batch_idx % 50 == 0 and batch_idx > 0:
            print(f"\n  Batch {batch_idx}: loss={loss.item():.4f}, grad_norm={grad_norm:.4f}")
    
    scheduler.step()
    
    if nan_count > 5:
        print(f"\n❌ Epoch {epoch+1} failed, stopping training")
        break
    
    # ========== VALIDATION ==========
    model_clip.eval()
    all_preds, all_labels = [], []
    
    # Encode all text prompts once for zero-shot evaluation
    with torch.no_grad():
        text_tokens_all = clip.tokenize(birads_prompts).to(device)
        text_features_all = model_clip.encode_text(text_tokens_all)
        text_features_all = text_features_all / (text_features_all.norm(dim=-1, keepdim=True) + 1e-8)
    
    with torch.no_grad():
        for images, texts, labels in val_loader_5class:
            images = images.to(device).float()
            labels = labels.to(device)
            
            # Encode images
            image_features = model_clip.encode_image(images)
            
            # Skip if NaN
            if torch.isnan(image_features).any():
                print("⚠ NaN in validation batch")
                continue
            
            # Normalize
            image_features = image_features / (image_features.norm(dim=-1, keepdim=True) + 1e-8)
            
            # Compare with ALL text prompts (zero-shot style)
            logit_scale = model_clip.logit_scale.exp().clamp(max=100)
            logits = logit_scale * image_features @ text_features_all.T
            
            # Predict class with highest similarity
            preds = logits.argmax(dim=1)
            
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())
    
    if len(all_preds) == 0:
        print(f"\n⚠ No valid predictions in epoch {epoch+1}")
        continue
    
    # Compute metrics
    acc = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
    avg_loss = train_loss / num_batches if num_batches > 0 else float('nan')
    
    print(f"\n📊 Epoch {epoch+1} Results:")
    print(f"   Loss: {avg_loss:.4f}")
    print(f"   Accuracy: {acc:.1f}%")
    print(f"   LR: {scheduler.get_last_lr()[0]:.2e}")
    print(f"   Valid batches: {num_batches}, NaN skipped: {nan_count}")
    
    # Save best model
    if acc > best_acc_base:
        best_acc_base = acc
        torch.save(model_clip.state_dict(), 'clip_full_imbalanced.pt')  # Save full model
        print(f"   ✓ Saved best model! (Acc: {best_acc_base:.1f}%)")

print(f"\n{'='*70}")
print(f"Training complete - Best accuracy: {best_acc_base:.1f}%")
print(f"{'='*70}")

# Final classification report
if len(all_preds) > 0:
    print("\n📊 Final Classification Report:")
    print(classification_report(
        all_labels, all_preds, 
        target_names=['BI-RADS 1', 'BI-RADS 2', 'BI-RADS 3', 'BI-RADS 4'],
        zero_division=0
    ))


TRAINING CLIP WITH CONTRASTIVE LOSS - ALL LAYERS TRAINABLE
✓ ALL layers trainable (visual + text encoder)
✓ Total trainable parameters: 151,277,313

Epoch 1/3


Training:  28%|██▊       | 52/186 [00:08<00:19,  6.92it/s]


  Batch 50: loss=1.4721, grad_norm=64.8021


Training:  55%|█████▍    | 102/186 [00:15<00:11,  7.21it/s]


  Batch 100: loss=1.2125, grad_norm=106.1445


...

Training:  55%|█████▍    | 102/186 [00:15<00:12,  6.54it/s]


  Batch 100: loss=1.0767, grad_norm=129.6267


Training:  82%|████████▏ | 152/186 [00:22<00:04,  7.25it/s]


  Batch 150: loss=0.8671, grad_norm=107.8156



📊 Epoch 3 Results:
   Loss: 0.9571
   Accuracy: 52.7%
   LR: 0.00e+00
   Valid batches: 186, NaN skipped: 0
   ✓ Saved best model! (Acc: 52.7%)

Training complete - Best accuracy: 52.7%

📊 Final Classification Report:
              precision    recall  f1-score   support

   BI-RADS 1       0.63      0.55      0.59        60
   BI-RADS 2       0.55      0.55      0.55        60
   BI-RADS 3       0.62      0.48      0.54        60
   BI-RADS 4       0.11      0.50      0.18         6

    accuracy                           0.53       186
   macro avg       0.48      0.52      0.47       186
weighted avg       0.58      0.53      0.55       186



# Actual run: With freezing

In [28]:
# Train Base Model - TASK VECTOR FRIENDLY (Only Visual Encoder)
print("="*70)
print("TRAINING CLIP - TASK VECTOR FRIENDLY")
print("FREEZING: Text encoder")
print("TRAINING: Visual encoder only")
print("="*70)

# === FREEZE TEXT ENCODER ===
for param in model_clip.transformer.parameters():
    param.requires_grad = False
for param in model_clip.token_embedding.parameters():
    param.requires_grad = False
for param in model_clip.ln_final.parameters():
    param.requires_grad = False

# Fix: positional_embedding is a single tensor, not a list
model_clip.positional_embedding.requires_grad = False

print("✓ Text encoder FROZEN")

# === UNFREEZE VISUAL ENCODER ===
for param in model_clip.visual.parameters():
    param.requires_grad = True

print("✓ Visual encoder TRAINABLE")

# Count parameters
trainable_params = sum(p.numel() for p in model_clip.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model_clip.parameters())
print(f"✓ Trainable: {trainable_params:,} / {total_params:,} ({100*trainable_params/total_params:.1f}%)")

# Optimizer - only visual encoder parameters
optimizer_base = optim.AdamW(
    [p for p in model_clip.visual.parameters() if p.requires_grad],
    lr=1e-6,  # Conservative LR
    weight_decay=1e-4,
    eps=1e-8
)

scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer_base, 
    T_max=CONFIG['epochs']
)

best_acc_base = 0

for epoch in range(CONFIG['epochs']):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1}/{CONFIG['epochs']}")
    print(f"{'='*70}")
    
    # ========== TRAINING ==========
    model_clip.train()
    # Keep text encoder in eval mode
    model_clip.transformer.eval()
    
    train_loss = 0
    num_batches = 0
    nan_count = 0
    
    for batch_idx, (images, texts, labels) in enumerate(tqdm(train_loader_5class, desc="Training", leave=False)):
        images = images.to(device).float()
        texts = texts.to(device)
        labels = labels.to(device)
        
        # Check inputs
        if torch.isnan(images).any() or torch.isinf(images).any():
            print(f"\n⚠ NaN/Inf in input images batch {batch_idx}")
            nan_count += 1
            continue
        
        optimizer_base.zero_grad()
        
        # Encode image (trainable)
        image_features = model_clip.encode_image(images)
        
        # Encode text (frozen, no grad)
        with torch.no_grad():
            text_features = model_clip.encode_text(texts)
        
        # Check for NaN after encoding
        if torch.isnan(image_features).any() or torch.isnan(text_features).any():
            print(f"\n⚠ NaN after encoding batch {batch_idx}")
            nan_count += 1
            if nan_count > 5:
                print("❌ Too many NaN batches, stopping epoch")
                break
            continue
        
        # Normalize features
        image_features = image_features / (image_features.norm(dim=-1, keepdim=True) + 1e-8)
        text_features = text_features / (text_features.norm(dim=-1, keepdim=True) + 1e-8)
        
        # Compute similarity (contrastive)
        logit_scale = model_clip.logit_scale.exp().clamp(max=100)
        logits_per_image = logit_scale * image_features @ text_features.T
        logits_per_text = logits_per_image.T
        
        # Contrastive loss
        batch_size = images.shape[0]
        target = torch.arange(batch_size).to(device)
        
        loss_img = nn.CrossEntropyLoss()(logits_per_image, target)
        loss_txt = nn.CrossEntropyLoss()(logits_per_text, target)
        loss = (loss_img + loss_txt) / 2
        
        # Check loss
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"\n⚠ NaN/Inf loss batch {batch_idx}")
            nan_count += 1
            continue
        
        loss.backward()
        
        # Gradient clipping
        grad_norm = torch.nn.utils.clip_grad_norm_(
            model_clip.visual.parameters(),
            max_norm=1.0
        )
        
        optimizer_base.step()
        
        train_loss += loss.item()
        num_batches += 1
        
        # Progress check
        if batch_idx % 50 == 0 and batch_idx > 0:
            print(f"\n  Batch {batch_idx}: loss={loss.item():.4f}, grad_norm={grad_norm:.4f}")
    
    scheduler.step()
    
    if nan_count > 5:
        print(f"\n❌ Epoch {epoch+1} failed, stopping training")
        break
    
    # ========== VALIDATION ==========
    model_clip.eval()
    all_preds, all_labels = [], []
    
    # Encode all text prompts once
    with torch.no_grad():
        text_tokens_all = clip.tokenize(birads_prompts).to(device)
        text_features_all = model_clip.encode_text(text_tokens_all)
        text_features_all = text_features_all / (text_features_all.norm(dim=-1, keepdim=True) + 1e-8)
    
    with torch.no_grad():
        for images, texts, labels in val_loader_5class:
            images = images.to(device).float()
            labels = labels.to(device)
            
            # Encode images
            image_features = model_clip.encode_image(images)
            
            if torch.isnan(image_features).any():
                print("⚠ NaN in validation batch")
                continue
            
            # Normalize
            image_features = image_features / (image_features.norm(dim=-1, keepdim=True) + 1e-8)
            
            # Compare with all text prompts
            logit_scale = model_clip.logit_scale.exp().clamp(max=100)
            logits = logit_scale * image_features @ text_features_all.T
            
            preds = logits.argmax(dim=1)
            
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())
    
    if len(all_preds) == 0:
        print(f"\n⚠ No valid predictions in epoch {epoch+1}")
        continue
    
    # Compute metrics
    acc = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
    avg_loss = train_loss / num_batches if num_batches > 0 else float('nan')
    
    print(f"\n📊 Epoch {epoch+1} Results:")
    print(f"   Loss: {avg_loss:.4f}")
    print(f"   Accuracy: {acc:.1f}%")
    print(f"   LR: {scheduler.get_last_lr()[0]:.2e}")
    print(f"   Valid batches: {num_batches}, NaN skipped: {nan_count}")
    
    # Save best model
    if acc > best_acc_base:
        best_acc_base = acc
        torch.save(model_clip.visual.state_dict(), 'clip_visual_imbalanced.pt')
        print(f"   ✓ Saved best visual encoder! (Acc: {best_acc_base:.1f}%)")

print(f"\n{'='*70}")
print(f"Training complete - Best accuracy: {best_acc_base:.1f}%")
print(f"{'='*70}")

# Final classification report
if len(all_preds) > 0:
    print("\n📊 Final Classification Report:")
    print(classification_report(
        all_labels, all_preds, 
        target_names=['BI-RADS 1', 'BI-RADS 2', 'BI-RADS 3', 'BI-RADS 4'],
        zero_division=0
    ))


TRAINING CLIP - TASK VECTOR FRIENDLY
FREEZING: Text encoder
TRAINING: Visual encoder only
✓ Text encoder FROZEN
✓ Visual encoder TRAINABLE
✓ Trainable: 88,111,361 / 151,277,313 (58.2%)

Epoch 1/3


Training:  27%|██▋       | 51/186 [00:07<00:20,  6.73it/s]


  Batch 50: loss=1.1265, grad_norm=169.1553


Training:  54%|█████▍    | 101/186 [00:14<00:12,  6.69it/s]


  Batch 100: loss=0.5116, grad_norm=60.4248


...

Training:  55%|█████▍    | 102/186 [00:13<00:09,  8.72it/s]


  Batch 100: loss=0.8712, grad_norm=147.4239


Training:  82%|████████▏ | 152/186 [00:20<00:04,  7.35it/s]


  Batch 150: loss=0.5316, grad_norm=157.7780



📊 Epoch 3 Results:
   Loss: 0.7345
   Accuracy: 55.4%
   LR: 0.00e+00
   Valid batches: 186, NaN skipped: 0
   ✓ Saved best visual encoder! (Acc: 55.4%)

Training complete - Best accuracy: 55.4%

📊 Final Classification Report:
              precision    recall  f1-score   support

   BI-RADS 1       0.59      0.67      0.62        60
   BI-RADS 2       0.55      0.52      0.53        60
   BI-RADS 3       0.67      0.50      0.57        60
   BI-RADS 4       0.12      0.33      0.17         6

    accuracy                           0.55       186
   macro avg       0.48      0.50      0.48       186
weighted avg       0.59      0.55      0.56       186



# Create task vector birads ( 1 to 5)

In [31]:
pretrained_sd = torch.load('clip_visual_pretrained.pt', weights_only=False)
finetuned_sd = torch.load('clip_visual_imbalanced.pt', weights_only=False)

# Create the vector dictionary: finetuned - pretrained
vector_dict = {}
for key in pretrained_sd.keys():
    if key in finetuned_sd and pretrained_sd[key].shape == finetuned_sd[key].shape:
        vector_dict[key] = finetuned_sd[key] - pretrained_sd[key]

print(f"✓ Created vector dict with {len(vector_dict)} parameter groups")


# Create TaskVector directly with the vector parameter
task_vector = TaskVector(vector=vector_dict)

print(task_vector)


✓ Created vector dict with 152 parameter groups


In [32]:
# Quick inspection
print(f"Task vector has {len(task_vector.vector)} parameter groups")
print(f"\nParameter names:")
for i, key in enumerate(list(task_vector.vector.keys())[:10]):  # First 10
    shape = task_vector.vector[key].shape
    print(f"  {i+1}. {key}: {shape}")
print("...")

# Pick one parameter to examine
sample_key = list(task_vector.vector.keys())[0]
sample_tensor = task_vector.vector[sample_key]
print(f"\nExample parameter '{sample_key}':")
print(f"  Shape: {sample_tensor.shape}")
print(f"  First 5 values: {sample_tensor.flatten()[:5].tolist()}")
print(f"  Mean change: {sample_tensor.mean().item():.8f}")


Task vector has 152 parameter groups

Parameter names:
  1. class_embedding: torch.Size([768])
  2. positional_embedding: torch.Size([50, 768])
  3. proj: torch.Size([768, 512])
  4. conv1.weight: torch.Size([768, 3, 32, 32])
  5. ln_pre.weight: torch.Size([768])
  6. ln_pre.bias: torch.Size([768])
  7. transformer.resblocks.0.attn.in_proj_weight: torch.Size([2304, 768])
  8. transformer.resblocks.0.attn.in_proj_bias: torch.Size([2304])
  9. transformer.resblocks.0.attn.out_proj.weight: torch.Size([768, 768])
  10. transformer.resblocks.0.attn.out_proj.bias: torch.Size([768])
...

Example parameter 'class_embedding':
  Shape: torch.Size([768])
  First 5 values: [-1.0201707482337952e-05, 2.5631568860262632e-05, -1.4337245374917984e-05, 1.498498022556305e-06, -1.2629665434360504e-05]
  Mean change: 0.00000606


# BIRADS 1 Focus 

In [33]:
# Create BI-RADS 1 focused dataset
print("="*70)
print("CREATING BI-RADS 1 FOCUSED DATASET")
print("="*70)

# Create binary labels: 1 = BI-RADS 1, 0 = Not BI-RADS 1
df_birads1_focus = df.copy()
df_birads1_focus['is_birads1'] = (df_birads1_focus['birads'] == 1).astype(int)

print(f"\nBinary label distribution:")
print(df_birads1_focus['is_birads1'].value_counts())
print(f"  BI-RADS 1: {(df_birads1_focus['is_birads1'] == 1).sum()}")
print(f"  Not BI-RADS 1: {(df_birads1_focus['is_birads1'] == 0).sum()}")

# Split data
train_df_birads1, val_df_birads1 = train_test_split(
    df_birads1_focus, 
    test_size=0.2, 
    stratify=df_birads1_focus['is_birads1'], 
    random_state=CONFIG['seed']
)

print(f"\n✓ Split: Train={len(train_df_birads1)}, Val={len(val_df_birads1)}")


CREATING BI-RADS 1 FOCUSED DATASET

Binary label distribution:
is_birads1
0    630
1    300
Name: count, dtype: int64
  BI-RADS 1: 300
  Not BI-RADS 1: 630

✓ Split: Train=744, Val=186


In [35]:
# Binary text prompts for BI-RADS 1 classification
birads1_prompts = [
    "a mammogram showing dense breast tissue, not BI-RADS category 1",
    "a mammogram showing BI-RADS category 1, entirely fatty breast tissue"
]

print(f"✓ Defined {len(birads1_prompts)} binary prompts for BI-RADS 1 classification")
print(f"  0: Not BI-RADS 1")
print(f"  1: BI-RADS 1")


✓ Defined 2 binary prompts for BI-RADS 1 classification
  0: Not BI-RADS 1
  1: BI-RADS 1


In [36]:
# Dataset class for BI-RADS 1 binary classification
class BIRADSBinaryDataset(Dataset):
    def __init__(self, dataframe, preprocess, text_prompts):
        self.df = dataframe.reset_index(drop=True)
        self.preprocess = preprocess
        self.text_tokens = clip.tokenize(text_prompts)
        print(f"✓ Tokenized {len(text_prompts)} binary text prompts")
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try:
            img = Image.open(row['image_path']).convert('RGB')
            img = self.preprocess(img)
            
            label = row['is_birads1']  # 0 or 1
            text = self.text_tokens[label]
            
            return img, text, label
        except Exception as e:
            print(f"Error loading {row['image_path']}: {e}")
            return None

# Create datasets
train_ds_birads1 = FilteredDataset(
    BIRADSBinaryDataset(train_df_birads1, preprocess=clip_preprocess, text_prompts=birads1_prompts)
)
val_ds_birads1 = FilteredDataset(
    BIRADSBinaryDataset(val_df_birads1, preprocess=clip_preprocess, text_prompts=birads1_prompts)
)

# Create DataLoaders
train_loader_birads1 = DataLoader(
    train_ds_birads1, 
    batch_size=CONFIG['batch_size'], 
    shuffle=True, 
    num_workers=2
)
val_loader_birads1 = DataLoader(
    val_ds_birads1, 
    batch_size=CONFIG['batch_size'], 
    num_workers=2
)

print(f"✓ BI-RADS 1 dataloaders ready: Train={len(train_ds_birads1)}, Val={len(val_ds_birads1)}")


✓ Tokenized 2 binary text prompts
✓ Valid samples: 744/744
✓ Tokenized 2 binary text prompts
✓ Valid samples: 186/186
✓ BI-RADS 1 dataloaders ready: Train=744, Val=186


In [37]:
# Train BI-RADS 1 Focused Model
print("="*70)
print("TRAINING CLIP - BI-RADS 1 FOCUSED")
print("="*70)

# Reload CLIP fresh (reset to pretrained state)
del model_clip
empty_cache()

model_clip, _ = clip.load(CONFIG['clip_model'], device=device)
model_clip = model_clip.float()

print(f"✓ Reloaded fresh CLIP {CONFIG['clip_model']}")

# Freeze text encoder
for param in model_clip.transformer.parameters():
    param.requires_grad = False
for param in model_clip.token_embedding.parameters():
    param.requires_grad = False
for param in model_clip.ln_final.parameters():
    param.requires_grad = False
model_clip.positional_embedding.requires_grad = False

# Unfreeze visual encoder
for param in model_clip.visual.parameters():
    param.requires_grad = True

print("✓ Text encoder FROZEN, Visual encoder TRAINABLE")

# Optimizer
optimizer_birads1 = optim.AdamW(
    [p for p in model_clip.visual.parameters() if p.requires_grad],
    lr=1e-6,
    weight_decay=1e-4,
    eps=1e-8
)

scheduler_birads1 = optim.lr_scheduler.CosineAnnealingLR(
    optimizer_birads1, 
    T_max=CONFIG['epochs']
)

best_acc_birads1 = 0

for epoch in range(CONFIG['epochs']):
    print(f"\n{'='*70}")
    print(f"Epoch {epoch+1}/{CONFIG['epochs']}")
    print(f"{'='*70}")
    
    # Training
    model_clip.train()
    model_clip.transformer.eval()
    
    train_loss = 0
    num_batches = 0
    nan_count = 0
    
    for batch_idx, (images, texts, labels) in enumerate(tqdm(train_loader_birads1, desc="Training BI-RADS 1", leave=False)):
        images = images.to(device).float()
        texts = texts.to(device)
        labels = labels.to(device)
        
        if torch.isnan(images).any():
            nan_count += 1
            continue
        
        optimizer_birads1.zero_grad()
        
        image_features = model_clip.encode_image(images)
        
        with torch.no_grad():
            text_features = model_clip.encode_text(texts)
        
        if torch.isnan(image_features).any():
            nan_count += 1
            if nan_count > 5:
                break
            continue
        
        image_features = image_features / (image_features.norm(dim=-1, keepdim=True) + 1e-8)
        text_features = text_features / (text_features.norm(dim=-1, keepdim=True) + 1e-8)
        
        logit_scale = model_clip.logit_scale.exp().clamp(max=100)
        logits_per_image = logit_scale * image_features @ text_features.T
        logits_per_text = logits_per_image.T
        
        batch_size = images.shape[0]
        target = torch.arange(batch_size).to(device)
        
        loss_img = nn.CrossEntropyLoss()(logits_per_image, target)
        loss_txt = nn.CrossEntropyLoss()(logits_per_text, target)
        loss = (loss_img + loss_txt) / 2
        
        if torch.isnan(loss):
            nan_count += 1
            continue
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_clip.visual.parameters(), max_norm=1.0)
        optimizer_birads1.step()
        
        train_loss += loss.item()
        num_batches += 1
    
    scheduler_birads1.step()
    
    # Validation
    model_clip.eval()
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        text_tokens_all = clip.tokenize(birads1_prompts).to(device)
        text_features_all = model_clip.encode_text(text_tokens_all)
        text_features_all = text_features_all / (text_features_all.norm(dim=-1, keepdim=True) + 1e-8)
    
    with torch.no_grad():
        for images, texts, labels in val_loader_birads1:
            images = images.to(device).float()
            labels = labels.to(device)
            
            image_features = model_clip.encode_image(images)
            
            if torch.isnan(image_features).any():
                continue
            
            image_features = image_features / (image_features.norm(dim=-1, keepdim=True) + 1e-8)
            
            logit_scale = model_clip.logit_scale.exp().clamp(max=100)
            logits = logit_scale * image_features @ text_features_all.T
            
            preds = logits.argmax(dim=1)
            
            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())
    
    if len(all_preds) == 0:
        continue
    
    acc = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
    avg_loss = train_loss / num_batches if num_batches > 0 else float('nan')
    
    print(f"\n📊 Epoch {epoch+1} Results:")
    print(f"   Loss: {avg_loss:.4f}")
    print(f"   Accuracy: {acc:.1f}%")
    print(f"   LR: {scheduler_birads1.get_last_lr()[0]:.2e}")
    
    if acc > best_acc_birads1:
        best_acc_birads1 = acc
        torch.save(model_clip.visual.state_dict(), 'clip_visual_birads1.pt')
        print(f"   ✓ Saved! (Best: {best_acc_birads1:.1f}%)")

print(f"\n{'='*70}")
print(f"BI-RADS 1 training complete - Best: {best_acc_birads1:.1f}%")
print(f"{'='*70}")

if len(all_preds) > 0:
    print("\n📊 Classification Report:")
    print(classification_report(
        all_labels, all_preds, 
        target_names=['Not BI-RADS 1', 'BI-RADS 1'],
        zero_division=0
    ))


TRAINING CLIP - BI-RADS 1 FOCUSED
✓ Reloaded fresh CLIP ViT-B/32
✓ Text encoder FROZEN, Visual encoder TRAINABLE

Epoch 1/3



📊 Epoch 1 Results:
   Loss: 1.3878
   Accuracy: 67.7%
   LR: 7.50e-07
   ✓ Saved! (Best: 67.7%)

Epoch 2/3



📊 Epoch 2 Results:
   Loss: 1.2665
   Accuracy: 74.2%
   LR: 2.50e-07
   ✓ Saved! (Best: 74.2%)

Epoch 3/3



📊 Epoch 3 Results:
   Loss: 1.2078
   Accuracy: 75.8%
   LR: 0.00e+00
   ✓ Saved! (Best: 75.8%)

BI-RADS 1 training complete - Best: 75.8%

📊 Classification Report:
               precision    recall  f1-score   support

Not BI-RADS 1       0.81      0.84      0.82       126
    BI-RADS 1       0.64      0.58      0.61        60

     accuracy                           0.76       186
    macro avg       0.72      0.71      0.72       186
 weighted avg       0.75      0.76      0.76       186



In [39]:

pretrained_sd = torch.load('clip_visual_pretrained.pt', weights_only=False)
birads1_sd = torch.load('clip_visual_birads1.pt', weights_only=False)

# Create vector dictionary
birads1_vector_dict = {}
for key in pretrained_sd.keys():
    if key in birads1_sd and pretrained_sd[key].shape == birads1_sd[key].shape:
        birads1_vector_dict[key] = birads1_sd[key] - pretrained_sd[key]

task_vector_birads1 = TaskVector(vector=birads1_vector_dict)

print(f"✓ BI-RADS 1 task vector created!")

# Quick inspection
total_params = sum(v.numel() for v in task_vector_birads1.vector.values())
print(f"✓ Total parameters in task vector: {total_params:,}")




✓ BI-RADS 1 task vector created!
✓ Total parameters in task vector: 87,849,216


In [40]:
print("="*70)
print("COMPARING BI-RADS 1 TASK VECTORS")
print("="*70)

# Check they have the same structure
print(f"\nStructure check:")
print(f"  BI-RADS 1-5 vector: {len(task_vector.vector)} parameters")
print(f"  BI-RADS 1 focus vector: {len(task_vector_birads1.vector)} parameters")
print(f"  Keys match: {set(task_vector.vector.keys()) == set(task_vector_birads1.vector.keys())}")


COMPARING BI-RADS 1 TASK VECTORS

Structure check:
  BI-RADS 1-5 vector: 152 parameters
  BI-RADS 1 focus vector: 152 parameters
  Keys match: True


# birads 5 focus

In [41]:
# BI-RADS 4 focused binary label
df_birads4_focus = df.copy()
df_birads4_focus['is_birads4'] = (df_birads4_focus['birads'] == 4).astype(int)

print("Binary label distribution:")
print(df_birads4_focus['is_birads4'].value_counts())

# Train/Val split
train_df_birads4, val_df_birads4 = train_test_split(
    df_birads4_focus,
    test_size=0.2,
    stratify=df_birads4_focus['is_birads4'],
    random_state=CONFIG['seed']
)


Binary label distribution:
is_birads4
0    900
1     30
Name: count, dtype: int64


In [42]:
birads4_prompts = [
    "a normal or benign mammogram, not BI-RADS category 4",  # Negative class
    "a mammogram showing BI-RADS category 4, suspicious abnormality"  # Positive class
]

print(f"✓ Defined {len(birads4_prompts)} prompts for BI-RADS 4 focus")


✓ Defined 2 prompts for BI-RADS 4 focus


In [43]:
class BIRADSBinaryDataset(Dataset):
    def __init__(self, dataframe, preprocess, text_prompts):
        self.df = dataframe.reset_index(drop=True)
        self.preprocess = preprocess
        self.text_tokens = clip.tokenize(text_prompts)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try:
            img = Image.open(row['image_path']).convert('RGB')
            img = self.preprocess(img)
            label = row['is_birads4']  # 0 or 1
            text = self.text_tokens[label]
            return img, text, label
        except Exception as e:
            print(f"Error loading {row['image_path']}: {e}")
            return None

train_ds_birads4 = FilteredDataset(
    BIRADSBinaryDataset(train_df_birads4, preprocess=clip_preprocess, text_prompts=birads4_prompts)
)
val_ds_birads4 = FilteredDataset(
    BIRADSBinaryDataset(val_df_birads4, preprocess=clip_preprocess, text_prompts=birads4_prompts)
)

train_loader_birads4 = DataLoader(train_ds_birads4, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader_birads4 = DataLoader(val_ds_birads4, batch_size=CONFIG['batch_size'], num_workers=2)

print(f"✓ DataLoaders ready: Train={len(train_ds_birads4)}, Val={len(val_ds_birads4)}")


✓ Valid samples: 744/744
✓ Valid samples: 186/186
✓ DataLoaders ready: Train=744, Val=186


In [44]:
# Reload CLIP (reset to pretrained)
del model_clip
empty_cache()
model_clip, _ = clip.load(CONFIG['clip_model'], device=device)
model_clip = model_clip.float()

# Freeze text encoder
for param in model_clip.transformer.parameters():
    param.requires_grad = False
for param in model_clip.token_embedding.parameters():
    param.requires_grad = False
for param in model_clip.ln_final.parameters():
    param.requires_grad = False
model_clip.positional_embedding.requires_grad = False

# Unfreeze visual encoder
for param in model_clip.visual.parameters():
    param.requires_grad = True

optimizer_birads4 = optim.AdamW(
    [p for p in model_clip.visual.parameters() if p.requires_grad],
    lr=1e-6, weight_decay=1e-4, eps=1e-8)

scheduler_birads4 = optim.lr_scheduler.CosineAnnealingLR(optimizer_birads4, T_max=CONFIG['epochs'])

best_acc_birads4 = 0

for epoch in range(CONFIG['epochs']):
    print(f"\n--- Epoch {epoch+1}/{CONFIG['epochs']} ---")
    model_clip.train(); model_clip.transformer.eval()
    train_loss, num_batches, nan_count = 0, 0, 0
    
    for images, texts, labels in tqdm(train_loader_birads4, desc="Training BI-RADS 4", leave=False):
        images, texts, labels = images.to(device).float(), texts.to(device), labels.to(device)
        if torch.isnan(images).any(): nan_count += 1; continue
        optimizer_birads4.zero_grad()

        image_features = model_clip.encode_image(images)
        with torch.no_grad():
            text_features = model_clip.encode_text(texts)
        if torch.isnan(image_features).any(): nan_count += 1; continue

        image_features = image_features / (image_features.norm(dim=-1, keepdim=True) + 1e-8)
        text_features = text_features / (text_features.norm(dim=-1, keepdim=True) + 1e-8)
        logit_scale = model_clip.logit_scale.exp().clamp(max=100)
        logits_per_image = logit_scale * image_features @ text_features.T
        logits_per_text = logits_per_image.T

        batch_size = images.shape[0]
        target = torch.arange(batch_size).to(device)
        loss_img = nn.CrossEntropyLoss()(logits_per_image, target)
        loss_txt = nn.CrossEntropyLoss()(logits_per_text, target)
        loss = (loss_img + loss_txt) / 2
        if torch.isnan(loss): nan_count += 1; continue

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_clip.visual.parameters(), max_norm=1.0)
        optimizer_birads4.step()

        train_loss += loss.item()
        num_batches += 1
    
    scheduler_birads4.step()

    # Validation
    model_clip.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        text_tokens_all = clip.tokenize(birads4_prompts).to(device)
        text_features_all = model_clip.encode_text(text_tokens_all)
        text_features_all = text_features_all / (text_features_all.norm(dim=-1, keepdim=True) + 1e-8)

        for images, texts, labels in val_loader_birads4:
            images, labels = images.to(device).float(), labels.to(device)
            image_features = model_clip.encode_image(images)
            if torch.isnan(image_features).any(): continue
            image_features = image_features / (image_features.norm(dim=-1, keepdim=True) + 1e-8)

            logit_scale = model_clip.logit_scale.exp().clamp(max=100)
            logits = logit_scale * image_features @ text_features_all.T
            preds = logits.argmax(dim=1)

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())
    
    if len(all_preds) == 0: continue
    acc = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
    avg_loss = train_loss / num_batches if num_batches > 0 else float('nan')
    print(f"Loss: {avg_loss:.4f}, Accuracy: {acc:.1f}%")
    if acc > best_acc_birads4:
        best_acc_birads4 = acc
        torch.save(model_clip.visual.state_dict(), 'clip_visual_birads4.pt')
        print(f"   ✓ Saved! (Best: {best_acc_birads4:.1f}%)")



--- Epoch 1/3 ---


Loss: 1.4063, Accuracy: 55.9%
   ✓ Saved! (Best: 55.9%)

--- Epoch 2/3 ---


Loss: 1.3762, Accuracy: 72.0%
   ✓ Saved! (Best: 72.0%)

--- Epoch 3/3 ---


Loss: 1.3643, Accuracy: 70.4%


In [45]:
# Create BI-RADS 4 Task Vector
pretrained_sd = torch.load('clip_visual_pretrained.pt', weights_only=False)
birads4_sd = torch.load('clip_visual_birads4.pt', weights_only=False)

# Create vector dictionary
birads4_vector_dict = {}
for key in pretrained_sd.keys():
    if key in birads4_sd and pretrained_sd[key].shape == birads4_sd[key].shape:
        birads4_vector_dict[key] = birads4_sd[key] - pretrained_sd[key]

task_vector_birads4 = TaskVector(vector=birads4_vector_dict)

print(f"✓ BI-RADS 4 task vector created!")

# Quick inspection
total_params = sum(v.numel() for v in task_vector_birads1.vector.values())
print(f"✓ Total parameters in task vector: {total_params:,}")


✓ BI-RADS 4 task vector created!
✓ Total parameters in task vector: 87,849,216


In [46]:
print("="*70)
print("COMPARING BI-RADS 1 TASK VECTORS")
print("="*70)

# Check they have the same structure
print(f"\nStructure check:")
print(f"  BI-RADS 1-5 vector: {len(task_vector.vector)} parameters")
print(f"  BI-RADS 1 focus vector: {len(task_vector_birads1.vector)} parameters")
print(f"  BI-RADS 4 focus vector: {len(task_vector_birads4.vector)} parameters")

print(f"  Keys match: {set(task_vector.vector.keys()) == set(task_vector_birads4.vector.keys())}")


COMPARING BI-RADS 1 TASK VECTORS

Structure check:
  BI-RADS 1-5 vector: 152 parameters
  BI-RADS 1 focus vector: 152 parameters
  BI-RADS 4 focus vector: 152 parameters
  Keys match: True


# Task Analogy

In [47]:
enhancement_vector_dict = {}
for key in task_vector_birads4.vector.keys():
    if key in task_vector_birads1.vector:
        enhancement_vector_dict[key] = task_vector_birads4.vector[key] - task_vector_birads1.vector[key]

print(f"✓ Enhancement vector has {len(enhancement_vector_dict)} parameter groups")

✓ Enhancement vector has 152 parameter groups


In [48]:
alpha = 0.25  # Adjust to test different strengths
print(f"✓ Using alpha = {alpha} for enhancement scaling")

✓ Using alpha = 0.25 for enhancement scaling


In [49]:
# Load base (pretrained) weights and BIRADS 1-5 task vector
base_sd = torch.load('clip_visual_pretrained.pt', weights_only=False)
birads_1to5_sd = torch.load('clip_visual_imbalanced.pt', weights_only=False)

# Add: base + (birads1-5 vector) + (alpha * enhancement vector)
final_sd = {}
for key in base_sd.keys():
    # Only update layers that exist in all vectors
    if key in birads_1to5_sd and key in enhancement_vector_dict:
        final_sd[key] = (
            base_sd[key]
            + (birads_1to5_sd[key] - base_sd[key])
            + alpha * enhancement_vector_dict[key]
        )
    elif key in birads_1to5_sd:
        # No enhancement for this layer, just use BIRADS 1-5 delta
        final_sd[key] = birads_1to5_sd[key]
    else:
        # No delta, use pretrained
        final_sd[key] = base_sd[key]

print(f"✓ Final synthesized weights created for model")


✓ Final synthesized weights created for model


In [50]:
# Load weights into a CLIP model
model_clip, _ = clip.load(CONFIG['clip_model'], device=device)
model_clip = model_clip.float()
model_clip.visual.load_state_dict(final_sd, strict=False)
print("✓ Synthesized model loaded into CLIP visual encoder")


✓ Synthesized model loaded into CLIP visual encoder


# Run test

In [51]:
# Load base (pretrained)
model_clip_base, _ = clip.load(CONFIG['clip_model'], device=device)
model_clip_base = model_clip_base.float()

# Imbalanced visual model
visual_imbalanced_sd = torch.load('clip_visual_imbalanced.pt', weights_only=False)
model_clip_imbalanced, _ = clip.load(CONFIG['clip_model'], device=device)
model_clip_imbalanced = model_clip_imbalanced.float()
model_clip_imbalanced.visual.load_state_dict(visual_imbalanced_sd, strict=False)

# Synthesized model: base + imbalanced + alpha*enhancement
synthesized_sd = final_sd  # from your previous synthesis cell!
model_clip_synth, _ = clip.load(CONFIG['clip_model'], device=device)
model_clip_synth = model_clip_synth.float()
model_clip_synth.visual.load_state_dict(synthesized_sd, strict=False)


<All keys matched successfully>

In [52]:
from sklearn.metrics import classification_report

def run_inference(model_clip, val_loader, text_prompts, device):
    # Encode text prompts once
    model_clip.eval()
    with torch.no_grad():
        text_tokens = clip.tokenize(text_prompts).to(device)
        text_features = model_clip.encode_text(text_tokens)
        text_features = text_features / (text_features.norm(dim=-1, keepdim=True) + 1e-8)

    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, texts, labels in val_loader:
            images = images.to(device).float()
            labels = labels.to(device)
            image_features = model_clip.encode_image(images)
            image_features = image_features / (image_features.norm(dim=-1, keepdim=True) + 1e-8)
            logit_scale = model_clip.logit_scale.exp().clamp(max=100)
            logits = logit_scale * image_features @ text_features.T
            preds = logits.argmax(dim=1)

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(labels.cpu().tolist())
    return all_preds, all_labels


In [53]:
# Set up prompts for 4-class BI-RADS
birads_prompts = [
    "a mammogram showing BI-RADS category 1, normal breast tissue",
    "a mammogram showing BI-RADS category 2, benign findings",
    "a mammogram showing BI-RADS category 3, probably benign findings",
    "a mammogram showing BI-RADS category 4, suspicious abnormality"
]

# Imbalanced visual model
imbal_preds, imbal_labels = run_inference(model_clip_imbalanced, val_loader_5class, birads_prompts, device)
print("\nImbalanced Visual Model:")
print(classification_report(imbal_labels, imbal_preds, target_names=["BI-RADS 1", "BI-RADS 2", "BI-RADS 3", "BI-RADS 4"], zero_division=0))

# Synthesized model
synth_preds, synth_labels = run_inference(model_clip_synth, val_loader_5class, birads_prompts, device)
print("\nSynthesized Model (Enhanced):")
print(classification_report(synth_labels, synth_preds, target_names=["BI-RADS 1", "BI-RADS 2", "BI-RADS 3", "BI-RADS 4"], zero_division=0))



Imbalanced Visual Model:
              precision    recall  f1-score   support

   BI-RADS 1       0.47      0.80      0.59        60
   BI-RADS 2       0.00      0.00      0.00        60
   BI-RADS 3       0.59      0.65      0.62        60
   BI-RADS 4       0.07      0.17      0.10         6

    accuracy                           0.47       186
   macro avg       0.28      0.40      0.33       186
weighted avg       0.34      0.47      0.39       186


Synthesized Model (Enhanced):
              precision    recall  f1-score   support

   BI-RADS 1       0.45      0.75      0.56        60
   BI-RADS 2       0.00      0.00      0.00        60
   BI-RADS 3       0.57      0.65      0.61        60
   BI-RADS 4       0.06      0.17      0.08         6

    accuracy                           0.46       186
   macro avg       0.27      0.39      0.31       186
weighted avg       0.33      0.46      0.38       186

